In [27]:
from pulp import *
import pandas as pd
import numpy as np

In [28]:
df_warehouse_cost=pd.read_excel("C:\\Users\\LXQMI\\Desktop\\Exhibits.xlsx",sheet_name='Exhibit 7.3')
df_annual_demand=pd.read_excel("C:\\Users\\LXQMI\\Desktop\\Exhibits.xlsx",sheet_name='Exhibit 7.4')
df_distribution_cost=pd.read_excel("C:\\Users\\LXQMI\\Desktop\\Exhibits.xlsx",sheet_name='Exhibit 7.5')
#df_warehouse_cost
#df_annual_demand
#df_distribution_cost

In [29]:
prob=LpProblem("Problem_HW5",LpMinimize)

# Constants in df_warehouse_cost
warehouse = df_warehouse_cost.iloc[2:10,1].tolist() # 8 potential warehouse locations
#print(warehouse)
F_C = df_warehouse_cost.iloc[2:10,2].tolist() # Fixed cost for each warehouse
#print(FC)
FC = dict(zip(warehouse, F_C))
V_C = df_warehouse_cost.iloc[2:10,3].tolist() # Variable cost for each warehouse
#print(VC)
VC = dict(zip(warehouse, V_C))
L = df_warehouse_cost.iloc[2:10,4].tolist() # Minimum capacity for each warehouse
#print(L)
min_cap = dict(zip(warehouse, L))
U = df_warehouse_cost.iloc[2:10,5].tolist() # Maximum capacity for each warehouse
#print(U)
max_cap = dict(zip(warehouse, U))

# Constants in df_annual_demand
cities = df_annual_demand.iloc[2:24,1].tolist() # 22 cities
#print(cities)
N = df_annual_demand.iloc[2:24,2].tolist() # Need of each city
#print(N)
Demand = dict(zip(cities, N))

# Constants in df_distribution_cost
D = df_distribution_cost.iloc[2:,1:].values # Distribution cost from warehouse i to city j
D = D.transpose().tolist() #transpose
#print(D)
Distribution_cost = makeDict([warehouse, cities], D)

In [30]:
# Decision Variables
X = LpVariable.dicts("Product_Shipped", (warehouse,cities), lowBound=0, cat = "Continous") # Products shipped to 22 cities from each warehouse
Y = LpVariable.dicts("Warehouse_Open", warehouse, cat="Binary") # Open warehouse or not
Z = LpVariable.dicts("Product_Held", warehouse, lowBound=0, cat = "Continous") # Total amount of product held in each warehouse

# Objective Function
fixed_cost = lpSum(FC[i] * Y[i]for i in warehouse)
variable_cost = lpSum((Z[i]-min_cap[i]*Y[i])*VC[i] for i in warehouse)
leasing_cost = fixed_cost + variable_cost
distribution_cost = lpSum(X[i][j]*Distribution_cost[i][j] for i in warehouse for j in cities)
prob += leasing_cost + distribution_cost

# Constraints
for i in warehouse:
    prob += lpSum(X[i][j] for j in cities) == Z[i] # Total volume of a single warehouse is equal to the sum of all the product it sends
    prob += Z[i] <= max_cap[i]*Y[i] # Warehouse capacity constraints
for j in cities:
    prob += lpSum([X[i][j] for i in warehouse]) >= Demand[j] # Meet the demand for each city

In [32]:
# Solve the problem
prob.solve()
print("Status:",LpStatus[prob.status])

# Results
for i in prob.variables():
    print(i.name, "=", i.varValue)

Status: Optimal
Product_Held_Atlanta = 1065.0
Product_Held_Buffalo = 1090.0
Product_Held_Chicago = 0.0
Product_Held_Cincinnati = 860.0
Product_Held_Detroit = 0.0
Product_Held_Pittsburgh = 0.0
Product_Held_Richmond = 560.0
Product_Held_St._Louis = 1500.0
Product_Shipped_Atlanta_Atlanta = 275.0
Product_Shipped_Atlanta_Birmingham = 160.0
Product_Shipped_Atlanta_Buffalo = 0.0
Product_Shipped_Atlanta_Charlotte = 135.0
Product_Shipped_Atlanta_Charteston = 0.0
Product_Shipped_Atlanta_Chattanooga = 160.0
Product_Shipped_Atlanta_Chicago = 0.0
Product_Shipped_Atlanta_Cincinnati = 0.0
Product_Shipped_Atlanta_Cleveland = 0.0
Product_Shipped_Atlanta_Columbus = 0.0
Product_Shipped_Atlanta_Detroit = 0.0
Product_Shipped_Atlanta_Evansville = 0.0
Product_Shipped_Atlanta_Ft_Wayne = 0.0
Product_Shipped_Atlanta_Indianapolis = 0.0
Product_Shipped_Atlanta_Knoxville = 125.0
Product_Shipped_Atlanta_Louisville = 0.0
Product_Shipped_Atlanta_Memphis = 0.0
Product_Shipped_Atlanta_Nashville = 210.0
Product_Shipped_

In [34]:
print("The optimal cost for is ", value(prob.objective))

The optimal cost for is  56075.0
